In [15]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import fitz
from io import BytesIO
import string
import pandas as pd
import pickle
import time

In [2]:
k = 9

In [3]:
url = "https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=posglos&NrKadencji=9"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, "html.parser")

# Find all links containing "IdDnia" in the href attribute
links = soup.find_all("a", href=lambda href: href and "IdDnia" in href)


clean_links = []
for link in links:
    clean_links.append('https://www.sejm.gov.pl/sejm9.nsf/' + link['href'])

In [4]:
clean_links

['https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1970',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1971',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1972',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1969',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1966',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1967',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1968',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1964',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1965',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1961',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1962',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1963',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=listaglos&IdDnia=1957',

In [5]:
clean_links_1 = []
for url in clean_links:    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    links = soup.find_all("a", href=lambda href: href and "NrGlosowania" in href)

    for link in links:
        clean_links_1.append('https://www.sejm.gov.pl/sejm9.nsf/' + link['href'])

clean_links_1 = list(set(clean_links_1))  

In [6]:
clean_links_1

['https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=5&NrGlosowania=22',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=72&NrGlosowania=54',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=43&NrGlosowania=34',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=38&NrGlosowania=13',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=39&NrGlosowania=14',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=35&NrGlosowania=47',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=73&NrGlosowania=170',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=6&NrGlosowania=539',
 'https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=63&NrGlosowan

In [7]:
len(clean_links_1)

9405

In [13]:
with open('clean_links_1.pkl', 'wb') as f:
    pickle.dump(clean_links_1, f)

In [26]:
# vote_results_links = []
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }
# headers={'User-Agent': 'Chrome/35.0.1916.47'}
r = requests.get(url, headers=headers)
for url in tqdm(clean_links_1[6438:]):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    a = soup.find("a", text="Wyniki indywidualne")
    if a:
        vote_results_links.append(a['href'])
    else:
        print('sus url', url)
    time.sleep(1)

  0%|▎                                                                             | 10/2967 [00:14<1:09:32,  1.41s/it]

sus url https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=8&NrGlosowania=11


  4%|███▍                                                                         | 133/2967 [03:09<1:05:39,  1.39s/it]

sus url https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=8&NrGlosowania=16


 27%|█████████████████████                                                          | 793/2967 [19:02<51:17,  1.42s/it]

...

 53%|█████████████████████████████████████████▌                                    | 1579/2967 [38:00<31:48,  1.37s/it]

sus url https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=8&NrGlosowania=1


 61%|███████████████████████████████████████████████▋                              | 1812/2967 [43:39<31:02,  1.61s/it]

sus url https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=8&NrGlosowania=10


100%|████████████████████████████████████████████████████████████████████████████| 2967/2967 [1:12:12<00:00,  1.46s/it]


In [27]:
len(vote_results_links)

9388

In [17]:
len(vote_results_links)

6428

In [28]:
with open('vote_results_links.pkl', 'wb') as f:
    pickle.dump(vote_results_links, f)

https://www.sejm.gov.pl/sejm9.nsf/agent.xsp?symbol=glosowania&NrKadencji=9&NrPosiedzenia=8&NrGlosowania=6

leads to 

https://orka.sejm.gov.pl/Glos9.nsf/nazwa/8_6/$file/GLOS_8_6.PDF

XDD

In [34]:
# df = pd.DataFrame()

headers={'User-Agent': 'Chrome/35.0.1916.47'}
# headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

for pdf_url in tqdm(vote_results_links[90:]):
#     print(pdf_url)
    response = requests.get(url=pdf_url, headers=headers, timeout=120)
    on_fly_mem_obj = BytesIO(response.content)
    doc = fitz.open(stream=on_fly_mem_obj, filetype="pdf")
    
    all_text = ""
    for page in doc:
        all_text += '\n' + page.get_text()

        
    parties = re.findall("\n\w+.+\(\d+\)", all_text)
    clean_parties = [re.sub("\n", '', party) for party in parties]
    clean_parties = [re.sub("\(\d+\)", '', party) for party in clean_parties]

    
    x = re.split("\n\w+.+\(\d+\)", all_text)
    
    posiedzenie_id = re.findall("posiedzenie \d+", x[0].lower())[0]
    glosowanie_id = re.findall("sowanie nr \d+", x[0])[0]
    data_time = re.findall("\d+-\d+-\d+ \d+:\d+:\d+", x[0])[0]
    posiedzenie_id = re.findall("\d+", posiedzenie_id)[0]
    glosowanie_id = re.findall("\d+", glosowanie_id)[0]

    
    for i in range(len(clean_parties)):
        party = clean_parties[i].strip()
        data_string = x[i+1]
#         data_string = re.sub("\nGŁOSOWAŁO - \d+ \nZA - \d+ \nPRZECIW - \d+ \nWSTRZYMAŁO SIĘ - \d+ \nNIE GŁOSOWAŁO - \d+ \n", '', data_string)
        lines = data_string.strip().split('\n')

        # Extract names and votes
        names = []
        votes = []
        for line in lines:
            line = line.strip()
            line = re.sub("Posiedzenie.+", '', line)
            v = re.search("[za|pr\.|ng\.|ws\.]+", line)
            if not re.search("\d", line) and v:
                votes.append(v.group())    
            if not re.search("\d", line) and re.search("\w[-|\s]\w", line):
                line = re.sub("[za|pr\.|ng\.|ws\.]+", '', line)
                names.append(line)
            
        df_sub = pd.DataFrame({
            'kadencja': k,
            'posiedzenie': posiedzenie_id,
            'numer': glosowanie_id,
            'data_godzina': data_time,
            'NazwiskoImie': names, 
            'Glos': votes,
            'Klub': party
        })
        df = pd.concat([df, df_sub], ignore_index=True)
        
        

  2%|█▍                                                                             | 175/9298 [00:59<51:23,  2.96it/s]


IndexError: list index out of range

In [38]:
vote_results_links[173]

'http://orka.sejm.gov.pl/Glos9.nsf/dok?OpenAgent&52_73'

In [36]:
df

,kadencja,posiedzenie,numer,data_godzina,NazwiskoImie,Glos,Klub
0,9,5,22,13-02-2020 20:09:51,ADAMCZYK ANDRZEJ,pr.,PiS
1,9,5,22,13-02-2020 20:09:51,GOŁOJUCH KAZIMIERZ,pr.,PiS
2,9,5,22,13-02-2020 20:09:51,MACIEREWICZ ANTONI,pr.,PiS
3,9,5,22,13-02-2020 20:09:51,SOIN AGNIESZKA,pr.,PiS
4,9,5,22,13-02-2020 20:09:51,ANDRUSZKIEWICZ ADAM,pr.,PiS
...,...,...,...,...,...,...,...
121871,9,5,4,13-02-2020 09:23:33,KORWIN-MIKKE JANUSZ,ng.,Konfederacja
121872,9,5,4,13-02-2020 09:23:33,KULESZA JAKUB,za,Konfederacja
121873,9,5,4,13-02-2020 09:23:33,BERKOWICZ KONRAD,ws.,Konfederacja
121874,9,5,4,13-02-2020 09:23:33,WINNICKI ROBERT,ws.,Konfederacja


In [39]:
df.to_csv('kadencja_9_scraped.csv', index=False)